In [4]:
from dependency import *
from my_tools import *

In [5]:
dep=get_dependency("fuzzing_result")

In [6]:
dep

[{'p11_constant_reverse': {'dep': [{'name': 'nick',
     'state': 'input',
     'type': 'int',
     'coord': 'infer.c:35:27'}],
   'ret': [{'coord': 'infer.c:40:9'}, {'coord': 'infer.c:42:9'}]},
  'p11_dict_new': {'dep': [],
   'ret': [{'coord': 'infer.c:17:13'}, {'coord': 'infer.c:20:5'}]}},
 {'p11_constant_reverse': {'dep': [{'name': 'nick',
     'state': 'input',
     'type': 'int',
     'coord': 'infer.c:35:27'}],
   'ret': [{'coord': 'infer.c:40:9'}, {'coord': 'infer.c:42:9'}]},
  'p11_dict_new': {'dep': [],
   'ret': [{'coord': 'infer.c:17:13'}, {'coord': 'infer.c:20:5'}]}}]

In [7]:
def read_file(filename):
    f=open(filename,"r")
    filelist=f.readlines()
    f.close()
    return filelist

In [8]:
def write_file(filename,code):
    f=open(filename,"w")
    f.write("".join(code))
    f.close()

In [4]:
#TODO instrument filelist
def instrument(dep):
    instrument_dict={}
    for e in dep:
        for func in e.values():
            if type(func)!=dict:
                continue
            for var in func["dep"]:
                filename=var["coord"].split(":")[0]
                filename=filename[4:]
                if filename not in instrument_dict:
                    instrument_dict[filename]=[]
                if var not in instrument_dict[filename]:
                    instrument_dict[filename].append(var)
    
    for filename in instrument_dict.keys():
        filelist=read_file(filename)
        for var in instrument_dict[filename]:
            varname=var["name"]
            state=var["state"]
            vartype=var["type"]
            line=int(var["coord"].split(":")[1])
            if "int" not in vartype:
                continue
            deref=""
            for l in vartype:
                if l=="*":
                    deref+="*"
            printf=f"fprintf(stderr,\"instrument: (line : {line}) {varname} : %d\\n\",{deref+varname});\n"
            if state=="var":
                filelist[line-1]=filelist[line-1][:-1]+printf
            elif state=="input":
                tmp=line-1
                while "{" not in filelist[tmp]:
                    tmp+=1
                filelist[tmp]=filelist[tmp][:-1]+printf
                
        write_file("instrumented_"+filename,filelist)
    return "instrumented_"+filename

In [5]:
def new_instrument(filelist,dep,name):
    instrument_dict={}
    for e in dep:
        for func in e.values():
            if type(func)!=dict:
                continue
            for var in func["dep"]:
                filename=var["coord"].split(":")[0]
                filename=filename[4:]
                if filename not in instrument_dict:
                    instrument_dict[filename]=[]
                if var not in instrument_dict[filename]:
                    instrument_dict[filename].append(var)
    for filename in instrument_dict.keys():
        #filelist=read_file(filename)
        for var in instrument_dict[filename]:
            varname=var["name"]
            state=var["state"]
            vartype=var["type"]
            line=int(var["coord"].split(":")[1])
            if "int" not in vartype:
                continue
            deref=""
            for l in vartype:
                if l=="*":
                    deref+="*"
            printf=f"fprintf(stderr,\"instrument: (line : {line}) {varname} : %d\\n\",{deref+varname});\n"
            if state=="var":
                filelist[line-1]=filelist[line-1][:-1]+printf
            elif state=="input":
                tmp=line-1
                while "{" not in filelist[tmp]:
                    tmp+=1
                filelist[tmp]=filelist[tmp][:-1]+printf
                
        write_file(name,filelist)
    

In [15]:
instrument(dep)

'instrumented_infer.c'

In [17]:
deref="*"
varname="a"
printf=f"printf(\"instrument: {deref+varname} : %d\\n\",{deref+varname})"

In [18]:
print(printf)

printf("instrument: *a: %d\n",*a)


In [6]:
filename="aa"
filename[-2]

'a'

In [ ]:
a